# Modelado y Evaluación

*Elaborado por:*
    
+ Javier Valencia Goujon, 123227
+ Mario Cruz García, 123808
+ Miguel Ávila del Bosque, 61100
+ Daniela Pinto Veizaga, 191471

**1. Paquetería Necesaria**

In [1]:
import sys
sys.path.append("./.local/lib/python3.6/site-packages")
import logging
import logging.config
logger = logging.getLogger()
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
fh = logging.FileHandler('example7.log')
fh.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
fh.setFormatter(formatter)
logger.addHandler(fh)
import pandas as pd
import numpy as np
from sklearn import svm, metrics, tree, decomposition, svm
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, ParameterGrid, RandomizedSearchCV, learning_curve, GridSearchCV
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc
import random
import pylab as pl
import matplotlib.pyplot as plt
from scipy import optimize
import time
import pandas as pd
from time import time
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

#from decorators import timer

**2. Modelos Seleccionados**
Para el desarrollo del modelado y ajuste de hiperparámetros se implementaron los siguientes modelos:

+ `Regresión Logística`
+ `Gradient Boosting Classifier`

Las secciones de esta fase es el diseño de prueba, la selección del modelo por medio de un benchmark y por último el ajuste de modelos. 

**3. Prueba de modelos**

*3.1. Importación de insumos*

+ Definimos nuestro conjunto de entrenamiento y nuestro conjunto de prueba.

> Conjunto de Entrenamiento. 

Este es producto de haber identificado el nùmero de transacciones que una determinada visita realiza en a cada departamento.

In [2]:
train = pd.read_csv('pre_train.csv')
train['TripType'] = train['TripType'].astype('category')
train

,Unnamed: 0,TripType,VisitNumber,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
0,1,3,106,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,3,121,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,3,153,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,3,162,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,3,164,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95669,95670,999,190811,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95670,95671,999,190843,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95671,95672,999,190888,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95672,95673,999,190930,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


> Conjunto de Prueba

Se aplica la misma transformaciòn que al conjunto de entrenamiento.

In [3]:

test = pd.read_csv('pre_test.csv')
test.head()

,Unnamed: 0,VisitNumber,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


> Esqueleto de entrega en Kaggle

In [4]:
sample = pd.read_csv('sample_submission.csv')

*3.2. Definimos nuestro y_train, x_train, y_test y x_train; estandarizamos los datos*

> Y_train

In [5]:
y_train = train['TripType']
y_train = pd.DataFrame(y_train).iloc[:,0]


>X_train

In [6]:
scaler = StandardScaler()

In [7]:
X_train = train
X_train = X_train.iloc[:,3:len(X_train.columns)]
scaled = pd.DataFrame(scaler.fit_transform(X_train.iloc[:,0:66]), columns = X_train.columns[0:66]).reset_index(drop = True)
X_train = pd.concat([scaled], axis = 1)
X_train = pd.DataFrame(X_train)
X_train

,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,BRAS & SHAPEWEAR,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
0,-0.046814,-0.074884,-0.149098,-0.219593,-0.107854,-0.230201,-0.099781,-0.074691,-0.0956,-0.09510,...,-0.114966,-0.016842,-0.237906,-0.05193,-0.18144,-0.1041,-0.138127,-0.062511,-0.157287,-0.102925
1,-0.046814,-0.074884,-0.149098,-0.219593,-0.107854,-0.230201,-0.099781,-0.074691,-0.0956,-0.09510,...,-0.114966,-0.016842,-0.237906,-0.05193,-0.18144,-0.1041,-0.138127,-0.062511,-0.157287,-0.102925
2,-0.046814,-0.074884,-0.149098,-0.219593,-0.107854,-0.230201,-0.099781,-0.074691,-0.0956,-0.09510,...,-0.114966,-0.016842,-0.237906,-0.05193,-0.18144,-0.1041,-0.138127,-0.062511,-0.157287,-0.102925
3,-0.046814,-0.074884,-0.149098,-0.219593,-0.107854,-0.230201,-0.099781,-0.074691,-0.0956,-0.09510,...,-0.114966,-0.016842,-0.237906,-0.05193,-0.18144,-0.1041,-0.138127,-0.062511,-0.157287,-0.102925
4,-0.046814,-0.074884,-0.149098,-0.219593,-0.107854,-0.230201,-0.099781,-0.074691,-0.0956,-0.09510,...,-0.114966,-0.016842,-0.237906,-0.05193,-0.18144,-0.1041,-0.138127,-0.062511,-0.157287,-0.102925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95669,-0.046814,-0.074884,-0.149098,-0.219593,-0.107854,-0.230201,-0.099781,-0.074691,-0.0956,4.37378,...,-0.114966,-0.016842,-0.237906,-0.05193,-0.18144,-0.1041,-0.138127,-0.062511,-0.157287,-0.102925
95670,-0.046814,-0.074884,-0.149098,-0.219593,-0.107854,-0.230201,-0.099781,-0.074691,-0.0956,-0.09510,...,-0.114966,-0.016842,-0.237906,-0.05193,-0.18144,-0.1041,-0.138127,-0.062511,-0.157287,-0.102925
95671,-0.046814,-0.074884,-0.149098,-0.219593,-0.107854,-0.230201,-0.099781,-0.074691,-0.0956,-0.09510,...,-0.114966,-0.016842,-0.237906,-0.05193,-0.18144,-0.1041,-0.138127,-0.062511,-0.157287,-0.102925
95672,-0.046814,-0.074884,-0.149098,-0.219593,-0.107854,-0.230201,-0.099781,-0.074691,-0.0956,-0.09510,...,-0.114966,-0.016842,-0.237906,-0.05193,-0.18144,-0.1041,-0.138127,-0.062511,-0.157287,-0.102925


> X_test

In [8]:
X_test = test
X_test = X_test.iloc[:,2:len(X_test.columns)]
X_test
X_test=X_test.dropna()
X_test = pd.DataFrame(scaler.fit_transform(X_test.iloc[:,0:66]), columns = X_test.columns[0:66]).reset_index(drop = True)
X_test

,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,BRAS & SHAPEWEAR,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
0,-0.044691,-0.082019,-0.151642,-0.218294,-0.118679,-0.237852,-0.104963,-0.078142,-0.100394,-0.091649,...,-0.12135,-0.016109,-0.235871,-0.05135,2.595369,-0.0988,-0.145535,-0.061406,-0.161081,-0.104266
1,-0.044691,-0.082019,-0.151642,2.696580,-0.118679,-0.237852,-0.104963,-0.078142,-0.100394,-0.091649,...,-0.12135,-0.016109,-0.235871,-0.05135,-0.180277,-0.0988,-0.145535,-0.061406,-0.161081,-0.104266
2,-0.044691,-0.082019,-0.151642,-0.218294,-0.118679,-0.237852,-0.104963,-0.078142,-0.100394,-0.091649,...,-0.12135,-0.016109,-0.235871,-0.05135,-0.180277,-0.0988,-0.145535,-0.061406,-0.161081,-0.104266
3,-0.044691,-0.082019,-0.151642,-0.218294,-0.118679,-0.237852,-0.104963,-0.078142,-0.100394,-0.091649,...,-0.12135,-0.016109,-0.235871,-0.05135,-0.180277,-0.0988,-0.145535,-0.061406,-0.161081,-0.104266
4,-0.044691,-0.082019,-0.151642,-0.218294,-0.118679,-0.237852,-0.104963,-0.078142,4.843041,-0.091649,...,-0.12135,-0.016109,-0.235871,-0.05135,-0.180277,-0.0988,-0.145535,-0.061406,-0.161081,-0.104266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95669,-0.044691,-0.082019,-0.151642,-0.218294,-0.118679,-0.237852,-0.104963,-0.078142,-0.100394,-0.091649,...,-0.12135,-0.016109,-0.235871,-0.05135,-0.180277,-0.0988,-0.145535,-0.061406,-0.161081,-0.104266
95670,-0.044691,-0.082019,2.506017,-0.218294,-0.118679,-0.237852,-0.104963,-0.078142,-0.100394,-0.091649,...,-0.12135,-0.016109,-0.235871,-0.05135,-0.180277,-0.0988,-0.145535,-0.061406,-0.161081,-0.104266
95671,-0.044691,-0.082019,-0.151642,-0.218294,-0.118679,-0.237852,-0.104963,-0.078142,-0.100394,-0.091649,...,-0.12135,-0.016109,-0.235871,-0.05135,-0.180277,-0.0988,-0.145535,-0.061406,-0.161081,-0.104266
95672,-0.044691,-0.082019,-0.151642,-0.218294,-0.118679,-0.237852,-0.104963,-0.078142,-0.100394,-0.091649,...,-0.12135,-0.016109,-0.235871,-0.05135,-0.180277,-0.0988,-0.145535,-0.061406,-0.161081,-0.104266


*3.3. Corrida de Modelos Seleccionados Aletoriamente*

> Regresión Logística

In [9]:
logistico = LogisticRegression(random_state= 1)
logistico.fit(X_train, y_train)
logit_result = logistico.predict_proba(X_test)
logistico.score(X_train, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.5774714133411376

> Clasificador Gradiente Boosting

In [10]:
GB = GradientBoostingClassifier(random_state=1)
GB.fit(X_train, y_train)
GB_result = GB.predict_proba(X_test)
GB.score(X_train, y_train)

0.603413675606748

**Resultado:** De los dos modelos seleccionados aleatoriamente, parece ser que la regresión logística es la mejor candidata...revismos y ajustemos sus parámetros.

**4. Selección de Hiperparámetros**

In [71]:
np.random.seed(seed=29123)
seed = 918739837
random.seed(21321)

In [72]:
def hiper_parametros():
    
    """"
        Devuelve un diccionario con los hiperparámetros
        y modelos empleados.
    """
    clfs = {
        'LR': LogisticRegression(penalty='l1', C=1e3, random_state=seed),
        'GB': GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=10, random_state=seed),
            }

    grid = { 
    'LR': { 'penalty': ['l1','l2'], 'C': [0.0001,0.001,0.01,0.1,1], 'random_state': [seed]},
    'GB': {'n_estimators': [1,10,25], 'learning_rate' : [0.001,0.01,0.05,0.1,0.5],'subsample' : [0.1,0.5,1.0], 'max_depth': [1,3,5,7], 'random_state': [seed]}
           }
    return clfs, grid

In [73]:
def magic_loop(models_to_run, clfs, grid, X_train_scaled, y_train, X_test_scaled, y_test, cv = 5,ParamTun = 'GridSearchCV'):
    """
        Ejecuta todos los modelos que se van a correr
        y lo hace ya sea utilizando GridSearchCV o RandomSearchCV. 
        Esta última elige aleatoriamente de entre los hiperparámetros
        que consideramos en la funcion define_hyper_param. Para 
        GridSearch se ejecuta también cross validation. Las métricas
        se guardan en una bitacora por medio de logging y ademas se 
        muestran en la salida de la terminal.
    """
    for n in range(1, 2):
        for index, clf in enumerate([clfs[x] for x in models_to_run]):
            logger.debug(models_to_run[index])
            print(models_to_run[index])
            parameter_values = grid[models_to_run[index]]
            try:
                if(ParamTun == 'GridSearchCV'):
                    grid_search = GridSearchCV(clf, parameter_values, cv=cv)
                    start = time()
                    if random.choice([True, False]):
                        modelo = grid_search.fit(X_train_scaled, y_train)
                        y_pred = modelo.predict(X_test_scaled)
                        scaled = True
                    else:
                        modelo = grid_search.fit(X_train, y_train)
                        y_pred = modelo.predict(X_test)
                        scaled = False

                    logger.debug("Best parameters set found on development set with GridSearchCV {}:".format(scaled))
                    logger.debug(" ")
                    logger.debug(modelo.best_params_)
                    logger.debug(" ")
                    logger.debug("Grid scores on development set:")
                    logger.debug(" ")
                    means = modelo.cv_results_['mean_test_score']
                    stds = modelo.cv_results_['std_test_score']
                    for mean, std, params in zip(means, stds, modelo.cv_results_['params']):
                        logger.debug("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
                    logger.debug(" ")

                    if models_to_run[index] != "LR":
                        print("dentro de accuracy con {}".format(models_to_run[index]))
                        logger.debug("Accuracy: {}".format(metrics.accuracy_score(y_test, y_pred)))
                        logger.debug(metrics.classification_report(y_pred, y_test))
                    else:
                        print("fuera de accuracy con {}".format(models_to_run[index]))
                        if scaled:
                            logger.debug("R2: {}".format(modelo.score(X_test_scaled, y_test))) ## Devuelve el R^2
                        else:
                            logger.debug("R2: {}".format(modelo.score(X_test, y_test))) ## Devuelve el R^2
                    logger.debug("grid_search took %.2f seconds" % (time() - start))

                    print("Best parameters set found on development set with GridSearchCV {}:".format(scaled))
                    print(" ")
                    print(modelo.best_params_)
                    print(" ")
                    print("Grid scores on development set:")
                    print(" ")
                    means = modelo.cv_results_['mean_test_score']
                    stds = modelo.cv_results_['std_test_score']
                    for mean, std, params in zip(means, stds, modelo.cv_results_['params']):
                        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
                    print(" ")

                    print("grid_search took %.2f seconds" % (time() - start))
                    if models_to_run[index] !="LR":
                        print("dentro de accuracy con {}".format(models_to_run[index]))
                        print("Accuracy: {}".format(metrics.accuracy_score(y_test, y_pred)))
                        print(metrics.classification_report(y_pred, y_test))
                    else:
                        print("fuera de accuracy con {}".format(models_to_run[index]))
                        if scaled:
                            print("R2: {}".format(modelo.score(X_test_scaled, y_test)))
                        else:
                            print("R2: {}".format(modelo.score(X_test, y_test)))
                else:
                    start = time()
                    random_search = RandomizedSearchCV(clf, parameter_values)
                    if random.choice([True, False]):
                        modelo = random_search.fit(X_train_scaled, y_train)
                        y_pred = modelo.predict(X_test_scaled)
                        scaled = True
                    else:
                        modelo = random_search.fit(X_train, y_train)
                        y_pred = modelo.predict(X_test)
                        scaled = False

                    logger.debug("Best parameters set found on development set with RandomizedSearchCV {}:".format(scaled))
                    logger.debug(" ")
                    logger.debug(modelo.best_params_)
                    logger.debug(" ")
                    logger.debug("Grid scores on development set:")
                    logger.debug(" ")

                    if models_to_run[index] != "LR":
                        print("dentro de accuracy con {}".format(models_to_run[index]))
                        logger.debug("Accuracy: {}".format(metrics.accuracy_score(y_test, y_pred)))
                        logger.debug(metrics.classification_report(y_pred, y_test))
                    else:
                        print("fuera de accuracy con {}".format(models_to_run[index]))
                        if scaled:
                            logger.debug("R2: {}".format(modelo.score(X_test_scaled, y_test))) ## Devuelve el R^2
                        else:
                            logger.debug("R2: {}".format(modelo.score(X_test, y_test))) ## Devuelve el R^2
                    logger.debug("RandomizedSearchCV took %.2f seconds" % (time() - start))

                    print("Best parameters set found on development set with RandomizedSearchCV {}:".format(scaled))
                    print(" ")
                    print(modelo.best_params_)
                    print(" ")
                    print("Grid scores on development set:")
                    print(" ")

                    print("RandomizedSearchCV took %.2f seconds" % (time() - start))
                    if models_to_run[index] != "LR":
                        print("dentro de accuracy con {}".format(models_to_run[index]))
                        print("Accuracy: {}".format(metrics.accuracy_score(y_test, y_pred)))
                        print(metrics.classification_report(y_pred, y_test))
                    else:
                        print("fuera de accuracy con {}".format(models_to_run[index]))
                        if scaled:
                            print("R2: {}".format(modelo.score(X_test_scaled, y_test)))
                        else:
                            print("R2: {}".format(modelo.score(X_test, y_test)))
            except IndexError as e:
                print('Error:', e)
                continue

In [74]:
def main(): 
    
    
    """
    Gran función
    """

    
    
    clfs, grid = hiper_parametros()
    models_to_run_GS=['LR', 'GB']
    models_to_run_RS=['LR','GB']
    
    #Se carga la base de datos y se divide el set de entrenamiento en uno para entrenar y otro para validacion
    
    train = pd.read_csv('pre_train.csv')
    train['TripType'] = train['TripType'].astype('category')
    scaler = StandardScaler()
    X = train
    X = X.iloc[:,3:len(X.columns)]
    y = train['TripType']
    y = pd.DataFrame(y).iloc[:,0]
    X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=1273483)

    #Se escalan los datos de entrenamiento
    scaled = pd.DataFrame(scaler.fit_transform(X_train.iloc[:,0:66]), columns = X_train.columns[0:66]).reset_index(drop = True)
    X_train = pd.concat([scaled], axis = 1)
    X_train = pd.DataFrame(X_train)

    
    #Se escalan los datos de validacion
    scaled = pd.DataFrame(scaler.fit_transform(X_val.iloc[:,0:66]), columns = X_val.columns[0:66]).reset_index(drop = True)
    X_val = pd.concat([scaled], axis = 1)
    X_val = pd.DataFrame(X_val)

    #Se corre el bucle con RandomSearch
    magic_loop(models_to_run_RS, clfs, grid, X_train, y_train, X_val, y_val, cv = 5, ParamTun = 'RandomSearchCV')
    #Se corre el bucle con GridSearch
    magic_loop(models_to_run_GS, clfs, grid, X_train, y_train, X_val, y_val, cv = 5, ParamTun = 'GridSearchCV')

In [ ]:
if __name__ == '__main__':
    main()

LR


/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning

Los resultados se encuentran en los archivos .txt anexos

# 5. Evaluación 

5.1, 5.2 Evaluación del modelo final con conjunto de prueba y reentrenamiento con hiperparámetros optimizados

Con los parámetros que tuvierón una mejor precisión se corrieron los mejores modelos.

In [ ]:
logistico = LogisticRegression(random_state= 918739837, penalty='l1', C= 1)
logistico.fit(X_train, y_train)
logit_result = logistico.predict_proba(X_test)
logistico.score(X_train, y_train)

In [ ]:
GB = GradientBoostingClassifier(random_state=918739837)
GB.fit(X_train, y_train)
GB_result = GB.predict_proba(X_test)
GB.score(X_train, y_train)

Se observa que los mejores modelos son el GBoost y la regresión lógistica; posiblemente porque estos captan la relación no aleatoria entre el tipo de visita y los patrones de consumo

In [ ]:
for columna in range(logit_result.shape[1]):
    sample.iloc[:,columna+1] = logit_result[:,columna]
sample.to_csv('pre_logit.csv', index=False)

In [ ]:
for columna in range(GB_result.shape[1]):
    sample.iloc[:,columna+1] = GB_result[:,columna]
sample.to_csv('pre_GB.csv', index=False)